In [2]:
# wind_analysis.ipynb - Cell 1: 環境設置與模組導入

# 1. 核心函式庫導入
from compas_notebook.viewer import Viewer
from compas.geometry import Point, Line, Polyline, Vector
from compas.colors import Color
from compas.datastructures import Mesh
import math
import numpy as np
from building_generator import create_dna_tower
import pandas as pd # 用於數據處理

# 2. 導入幾何模組
# 找到 building_generator.py
try:
    from building_generator import create_single_twisted_prism
    from visualizer_utils import draw_best_model
    print("✅")
except ModuleNotFoundError:
    print("❌ ")

# 3. 初始化 Viewer
viewer = Viewer()
viewer.scene.clear()

# 4. 定義 CFD 參數 
BASE_X = 4.0   # 矩形底面的 X 寬度
BASE_Y = 4.0   # 矩形底面的 Y 深度
HEIGHT = 10.0  # 總高度 

U = 1.0        # 遠場風速
R_OBS = max(BASE_X, BASE_Y) * 0.55 # 障礙物等效半徑
COLOR_SLOW = Color.red
COLOR_FAST = Color.blue

# 風場範圍
Z_RANGE = (0.0, HEIGHT)
Y_RANGE = (-3.0 * BASE_Y, 3.0 * BASE_Y)

❌ 


In [3]:
# wind_analysis.ipynb - Cell 2: CFD 核心邏輯定義 (所有函式整合)

# 全域變數用於儲存流線數據
segments = []
wind_vectors = []

# --- 1. 速度場函式 ---
def velocity_field(p: Point, R_obs: float, angle_deg: float, U: float) -> Vector:
    angle_rad = math.radians(-angle_deg) 
    
    x_rot = p.x * math.cos(angle_rad) - p.y * math.sin(angle_rad)
    y_rot = p.x * math.sin(angle_rad) + p.y * math.cos(angle_rad)
    
    r_sq = x_rot*x_rot + y_rot*y_rot
    U_inf = U 
    
    if r_sq < R_obs * R_obs * 0.9: 
        u_x_rot = 0.0
        u_y_rot = 0.0
    else:
        u_x_rot = U_inf * (1 - R_obs*R_obs / r_sq) 
        u_y_rot = U_inf * (2 * R_obs*R_obs / r_sq) * (x_rot * y_rot / r_sq) 
    
    u_x = u_x_rot * math.cos(angle_rad) + u_y_rot * math.sin(angle_rad)
    u_y = u_y_rot * math.cos(angle_rad) - u_x_rot * math.sin(angle_rad)
    
    return Vector(u_x, u_y, 0.0)


# --- 2. RK2 函式 ---
def rk2(p: Point, dt: float, R_obs: float, angle_deg: float, U: float) -> Point:
    h = dt
    v1 = velocity_field(p, R_obs, angle_deg, U)
    k1 = Vector(v1.x * h, v1.y * h, 0.0)
    
    p_mid = p + k1 * 0.5
    v2 = velocity_field(p_mid, R_obs, angle_deg, U)
    k2 = Vector(v2.x * h, v2.y * h, 0.0)
    
    return p + k2


# --- 3. 流線生成函式 ---
def add_streamline(p_start: Point, R_obs: float, angle_deg: float, U: float, Y_RANGE: tuple, 
                   stream_len: int = 200, dt: float = 0.05):
    
    current_p = p_start
    
    for _ in range(stream_len):
        
        if current_p.y < Y_RANGE[0] or current_p.y > Y_RANGE[1]:
            break
        
        next_p = rk2(current_p, dt, R_obs, angle_deg, U) 
        v = velocity_field(current_p, R_obs, angle_deg, U)
        
        if -10.0 < current_p.x < 10.0 and Y_RANGE[0] < current_p.y < Y_RANGE[1]:
            wind_vectors.append(v)
            
        current_p = next_p


# --- 4. 評分函式 ---
def compute_wind_score(wind_vectors: list, U_inf: float, rho: float = 1.2):
    if not wind_vectors:
        return {"avg_speed": 0.0, "momentum_deficit": 0.0, "score": 0.0}
        
    speeds = np.array([v.length for v in wind_vectors], dtype=float)
    deficit = rho * np.sum((U_inf - speeds) ** 2)
    
    Deficit_MAX = 50.0 
    score_norm = 1.0 - (deficit / Deficit_MAX)
    score = float(np.clip(score_norm * 100.0, 0.0, 100.0))
    
    return {
        "avg_speed": float(np.mean(speeds)),
        "momentum_deficit": float(deficit),
        "score": score
    }

In [4]:
def draw_wind_analysis(angle: float, model_mesh, viewer, 
                       R_obs: float, U_inf: float, Y_RANGE: tuple, 
                       wind_vectors: list, segments: list, 
                       HEIGHT: float,
                       NUM_LINES: int = 10, STREAM_LEN: int = 200, DT: float = 0.05):
    
    viewer.scene.clear()
    
    # 1. 繪製建築物
    viewer.scene.add(model_mesh, facecolor=Color.white, opacity=0.8)
    
    # 2. 生成流線和向量
    segments.clear()
    wind_vectors.clear()
    
    y_starts = np.linspace(Y_RANGE[0], Y_RANGE[1], NUM_LINES)
    z_mid = HEIGHT * 0.5 
    
    # 計算並繪製流線/向量
    for y_start in y_starts:
        p_start = Point(-10.0, y_start, z_mid)
        current_p = p_start
        
        for _ in range(STREAM_LEN):
            if current_p.y < Y_RANGE[0] or current_p.y > Y_RANGE[1]:
                break
            
            # 使用 rk2 進行積分
            next_p = rk2(current_p, DT, R_obs, angle, U_inf) 
            
            # 儲存流線段供繪圖
            segments.append(Line(current_p, next_p)) 
            
            # 評分
            if -10.0 < current_p.x < 10.0 and Y_RANGE[0] < current_p.y < Y_RANGE[1]:
                # 使用 velocity_field 獲取速度
                v = velocity_field(current_p, R_obs, angle, U_inf)
                wind_vectors.append((current_p, v)) 
                
            current_p = next_p

    # 3. 風速箭頭顏色定義
    
    # 繪製風速向量 (從 wind_vectors 列表繪圖)
    for p, v in wind_vectors:
        speed = v.length
        
        # 顏色邏輯：與遠場風速 U_inf 比較
        if speed > U_inf * 1.05: # 加速區 -> 藍色
            color = COLOR.BLUE 
        elif speed < U_inf * 0.95: # 減速區 -> 紅色
            color = COLOR.RED 
        else:
            color = Color.green # 中間風速 -> 綠色
        
        # 繪製向量箭頭 (縮小向量 v 以便視覺化)
        viewer.scene.add(Line(p, p + v * 0.5), color=color, linewidth=2.0) 


    # 4. 繪製流線段
    viewer.scene.add(segments, color=Color.black, linewidth=0.5)

    # 5. 輸出視覺化
    viewer.scene.update_bounds()
    viewer.scene.scale_to_bounds(0.5) 
    
    viewer.draw()

In [5]:
import pandas as pd
import numpy as np
import math
from building_generator import create_single_twisted_prism # 確保已載入修正後的版本

# ==============================================================================
# 參數定義 (請確保這些參數與 Cell 2 中的定義一致)
# ==============================================================================
BASE_X = 20.0  # 建築基底 X 寬度
BASE_Y = 20.0  # 建築基底 Y 寬度
HEIGHT = 100.0 # 建築高度
U = 10.0       # 迎風側參考風速 (m/s)
R_OBS = 200.0  # 觀察區域半徑 (m)
Y_RANGE = 2.0  # 高度 (m)

# 模擬結果儲存列表
results_data = []

# ==============================================================================
# 評分函式修正：產生細膩數值分佈
# ==============================================================================

def calculate_score(avg_speed: float, momentum_deficit: float, U: float) -> float:
    
    # --- 評分參數設定 ---
    V_COMFORT_MAX = 0.5 * U  # 舒適風速閾值 (例如，參考風速的 50%)
    V_CRITICAL = 0.8 * U     # 臨界風速 (風速超過此值懲罰力度大幅增加)
    
    # --- 1. 風速舒適度評分 (Score_V) ---
    
    # 使用平滑化函式來避免硬性截斷
    if avg_speed <= V_COMFORT_MAX:
        # 風速在舒適區內，給予高分
        score_v = 100.0
    else:
        # 風速超過舒適閾值，分數開始平滑下降
        
        # 懲罰因子：使用指數衰減，使分數平滑地從 V_COMFORT_MAX 降至 V_CRITICAL
        # 假設懲罰在 V_CRITICAL 處降到約 20 分 (20/100 = 0.2)
        penalty_factor = math.exp(-3.0 * (avg_speed - V_COMFORT_MAX) / U) 
        score_v = 100.0 * penalty_factor
        
        # 如果風速超過臨界值，分數迅速趨近於 0
        if avg_speed > V_CRITICAL:
            score_v = max(0.0, score_v * 0.2) 
            
    # --- 2. 動量虧損評分 (Score_M) ---
    
    # 假設動量虧損越小越好。將虧損值反向映射到 0-100 區間。
    # 假設理想虧損接近 0，最大虧損（例如 U*U/2）為 0 分。
    # 由於 momentum_deficit 單位為 (m/s)^2，我們用 U*U 來標準化。
    
    # 進行標準化和反向評分 (假設一個理想的動量虧損上限為 0.2 * U*U)
    M_MAX = 0.2 * U**2
    score_m = max(0.0, 100.0 * (1.0 - (momentum_deficit / M_MAX)))
    
    # --- 3. 總評分 (平衡權重) ---

    W_V = 0.6  # 風速權重 (60%)
    W_M = 0.4  # 動量虧損權重 (40%)
    
    final_score = (W_V * score_v) + (W_M * score_m)
    
    # 確保分數在 0 到 100 之間
    return max(0.0, min(100.0, final_score))


# ... (保持模擬迴圈程式碼不變)

# ==============================================================================
# CFD 模擬和評分主迴圈
# ==============================================================================
# ... (保持 Cell 3 的 for loop 迴圈和模擬邏輯不變)

# 假設您 Cell 3 的主迴圈是這樣的：
print("--- 開始模型的 CFD 模擬 ---")
for angle_deg in np.arange(0, 360, 5):
    try:
        # 1. 生成 Mesh
        model_mesh = create_single_twisted_prism(angle_deg, BASE_X, BASE_Y, HEIGHT)

        # --- 模擬結果佔位符 (Placeholder) ---
        # 模擬扭轉角度對風速和動量虧損的影響
        # 假設風速在 45, 135, 225, 315 度時表現較好
        angle_rad = math.radians(angle_deg)
        
        # 模擬 Avg Speed：在特定角度會下降，其他角度較高
        avg_speed_sim = U * (0.5 + 0.4 * abs(math.sin(2 * angle_rad)))
        
        # 模擬 Momentum Deficit：與 Avg Speed 呈弱相關
        momentum_deficit_sim = U**2 * (0.1 + 0.05 * abs(math.cos(angle_rad)))
        # --- 模擬結果佔位符結束 ---

        # 3. 計算分數
        score = calculate_score(avg_speed_sim, momentum_deficit_sim, U)

        # 4. 儲存結果
        results_data.append({
            'Angle (deg)': angle_deg,
            'Avg Speed': avg_speed_sim,
            'Momentum Deficit': momentum_deficit_sim,
            'Score (0-100)': score
        })

    except Exception as e:
        # 處理幾何生成或模擬時的錯誤
        print(f"❌ 警告: 角度 {angle_deg}° 生成幾何時發生錯誤: {e}")
        # 如果失敗，添加 NaN 行
        results_data.append({
            'Angle (deg)': angle_deg,
            'Avg Speed': np.nan,
            'Momentum Deficit': np.nan,
            'Score (0-100)': np.nan
        })

print("--- 模擬完成 ---")

# ==============================================================================
# 結果分析和報告 (保持不變)
# ==============================================================================

# 創建 DataFrame
sorted_results = pd.DataFrame(results_data).sort_values(by='Score (0-100)', ascending=False)

# 報告分數最高的 Top 5
top_5_scores = sorted_results.head(5)
print("\n 最高評分 (Top 5) 角度報告:")
print(top_5_scores.to_markdown(index=False, floatfmt=".2f"))

# 報告分數最低的 Bottom 5 (僅顯示有分數的結果)
bottom_5_scores = sorted_results.dropna().tail(5)
print("\n 最低評分 (Bottom 5) 角度報告:")
print(bottom_5_scores.to_markdown(index=False, floatfmt=".2f"))

# 中位數和中間分數報告
median_score = sorted_results['Score (0-100)'].median()
print(f"\n 模擬結果中位數分數: {median_score:.2f}")

# 找到最接近中位數的 5 個分數 (Median 5)
if not sorted_results.dropna().empty:
    median_scores_abs_diff = abs(sorted_results.dropna()['Score (0-100)'] - median_score)
    median_5_scores = sorted_results.loc[median_scores_abs_diff.nsmallest(5).index]
else:
    median_5_scores = pd.DataFrame(columns=['Angle (deg)', 'Avg Speed', 'Momentum Deficit', 'Score (0-100)'])

print("\n 最接近中位數評分 (Median 5) 角度報告:")
print(median_5_scores.to_markdown(index=False, floatfmt=".2f"))

# 處理 Index Error (使用修正後的安全檢查)
if not top_5_scores.empty:
    best_angle = top_5_scores.iloc[0]['Angle (deg)']
    best_score = top_5_scores.iloc[0]['Score (0-100)']
    print(f"\n 最佳扭轉角度: {best_angle}° (評分: {best_score:.2f})")
else:
    print("\n 警告：無法確定最佳角度，因為所有模擬運行都失敗或分數為 NaN。")
    best_angle = 0.0
    best_score = 0.0

# 設置 sorted_results 供 Cell 4 使用 (保持在全域作用域)
# global sorted_results # 如果您在函式外部定義了 sorted_results，則不需要這行

--- 開始模型的 CFD 模擬 ---
--- 模擬完成 ---

 最高評分 (Top 5) 角度報告:
|   Angle (deg) |   Avg Speed |   Momentum Deficit |   Score (0-100) |
|--------------:|------------:|-------------------:|----------------:|
|         90.00 |        5.00 |              10.00 |           80.00 |
|        270.00 |        5.00 |              10.00 |           80.00 |
|          0.00 |        5.00 |              15.00 |           70.00 |
|        180.00 |        5.00 |              15.00 |           70.00 |
|        275.00 |        5.69 |              10.44 |           67.84 |

 最低評分 (Bottom 5) 角度報告:
|   Angle (deg) |   Avg Speed |   Momentum Deficit |   Score (0-100) |
|--------------:|------------:|-------------------:|----------------:|
|        145.00 |        8.76 |              14.10 |           15.69 |
|        330.00 |        8.46 |              14.33 |           15.58 |
|         30.00 |        8.46 |              14.33 |           15.58 |
|        150.00 |        8.46 |              14.33 |           15.58 

In [6]:
# wind_analysis.ipynb: 匯出最佳扭轉角

import json
import os

# 確保 best_angle 變數已經在 Cell 3 中計算出來
if 'best_angle' in locals() or 'best_angle' in globals():
    output_data = {
        'best_angle_deg': float(best_angle)
    }
    
    # 儲存到名為 best_angle.json 的檔案
    output_filename = 'best_angle_data.json'
    
    with open(output_filename, 'w') as f:
        json.dump(output_data, f)
        
    print(f"✅ 最佳扭轉角已成功匯出到檔案: {output_filename}")
    print(f"   數值: {best_angle:.2f} 度")
else:
    print("❌ 錯誤：找不到 best_angle 變數，請確保 Cell 3 已成功運行。")

✅ 最佳扭轉角已成功匯出到檔案: best_angle_data.json
   數值: 90.00 度
